In [1]:
import pandas as pd
import numpy as np
import json
import re
import warnings
from pathlib import Path
from datetime import datetime
import yaml

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Random seed: {RANDOM_SEED}")

✅ Libraries imported successfully!
Pandas version: 2.3.2
NumPy version: 2.2.6
Random seed: 42


In [2]:
# Load configuration
CONFIG_PATH = Path('../configs/config.yaml')

if CONFIG_PATH.exists():
    with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    print("✅ Configuration loaded from config.yaml")
else:
    # Default configuration
    config = {
        'data': {
            'raw_path': '../Data/',
            'processed_path': '../Data/processed/',
        },
        'cleaning': {
            'placeholder_tokens': ['none', 'null', 'nan', 'n/a', 'na', '-', '--', 'unknown', '?'],
            'price_min': 1000,  # Minimum valid price (1,000 Toman)
            'price_max': 1_000_000_000_000,  # Maximum valid price (1 Trillion Toman)
            'remove_zero_prices': True,
        },
        'random_seed': RANDOM_SEED
    }
    print("⚠️  Using default configuration")

# Create processed data directory
PROCESSED_PATH = Path(config['data']['processed_path'])
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Data paths:")
print(f"   Raw: {config['data']['raw_path']}")
print(f"   Processed: {config['data']['processed_path']}")

✅ Configuration loaded from config.yaml

📁 Data paths:
   Raw: Data/
   Processed: Data/processed/


In [5]:
# Define data path
DATA_PATH = Path(config['data']['raw_path'])

print("Loading datasets...")
print("-" * 80)

# Load all tables
base_products = pd.read_parquet('../Data/base_products.parquet')
members = pd.read_parquet('../Data/members.parquet')
searches = pd.read_parquet('../Data/searches.parquet')
base_views = pd.read_parquet('../Data/base_views.parquet')
final_clicks = pd.read_parquet('../Data/final_clicks.parquet')
shops = pd.read_parquet('../Data/shops.parquet')
categories = pd.read_parquet('../Data/categories.parquet')
brands = pd.read_parquet('../Data/brands.parquet')
cities = pd.read_parquet('../Data/cities.parquet')

print("✅ All datasets loaded successfully!")
print("-" * 80)

# Store original shapes for comparison later
original_shapes = {
    'base_products': base_products.shape,
    'members': members.shape,
    'searches': searches.shape,
    'base_views': base_views.shape,
    'final_clicks': final_clicks.shape,
    'shops': shops.shape,
    'categories': categories.shape,
    'brands': brands.shape,
    'cities': cities.shape
}

for name, shape in original_shapes.items():
    print(f"{name:20s}: {shape[0]:>10,} rows × {shape[1]:>3} columns")

Loading datasets...
--------------------------------------------------------------------------------
✅ All datasets loaded successfully!
--------------------------------------------------------------------------------
base_products       :  1,022,298 rows ×   8 columns
members             :  1,948,665 rows ×   4 columns
searches            :    588,347 rows ×   9 columns
base_views          :    199,916 rows ×   4 columns
final_clicks        :     17,371 rows ×   4 columns
shops               :     23,342 rows ×   4 columns
categories          :        746 rows ×   3 columns
brands              :      2,025 rows ×   2 columns
cities              :        651 rows ×   2 columns


In [18]:
def clean_text_field(series, placeholder_tokens=None, convert_to_none=True):
    """
    Clean and normalize text fields.
    
    Args:
        series: pandas Series to clean
        placeholder_tokens: list of strings to treat as missing (e.g., ['null', 'none', 'n/a'])
        convert_to_none: if True, convert empty/placeholder values to None; otherwise keep as empty string
    
    Returns:
        Cleaned pandas Series
    """
    if placeholder_tokens is None:
        placeholder_tokens = config['cleaning']['placeholder_tokens']
    
    # Convert to string type
    cleaned = series.astype('string')
    
    # Strip whitespace
    cleaned = cleaned.str.strip()
    
    # Remove multiple consecutive spaces
    cleaned = cleaned.str.replace(r'\s+', ' ', regex=True)
    
    # Handle placeholders
    placeholder_set = set([str(p).lower() for p in placeholder_tokens])
    mask_placeholder = cleaned.str.lower().isin(placeholder_set)
    
    if convert_to_none:
        # Convert empty strings and placeholders to None
        cleaned = cleaned.where(
            (cleaned.notna()) & (cleaned != '') & (~mask_placeholder),
            None
        )
    else:
        # Convert placeholders to empty string
        cleaned = cleaned.where(~mask_placeholder, '')
    
    return cleaned


def normalize_persian_text(series):
    """
    Normalize Persian/Arabic characters.
    - Convert Arabic 'ي' and 'ك' to Persian 'ی' and 'ک'
    - Remove zero-width characters
    - Normalize diacritics
    """
    if series is None or len(series) == 0:
        return series
    
    cleaned = series.astype('string')
    
    # Arabic to Persian character conversion
    cleaned = cleaned.str.replace('ي', 'ی', regex=False)  # Arabic Yeh to Persian Yeh
    cleaned = cleaned.str.replace('ك', 'ک', regex=False)  # Arabic Kaf to Persian Kaf
    
    # Remove zero-width characters (ZWJ, ZWNJ, etc.)
    # Keep ZWNJ (\u200c) as it's important for Persian text
    cleaned = cleaned.str.replace('[\u200b\u200d\u200e\u200f]', '', regex=True)
    
    # Remove diacritics (َ ُ ِ ً ٌ ٍ ّ ْ) for better text matching
    cleaned = cleaned.str.replace('[\u064B-\u0652]', '', regex=True)
    
    return cleaned


def normalize_english_text(series):
    """
    Normalize English text.
    - Convert to lowercase (optional)
    - Remove special characters (optional)
    """
    if series is None or len(series) == 0:
        return series
    
    cleaned = series.astype('string')
    
    # Convert to lowercase for consistency (optional)
    # cleaned = cleaned.str.lower()
    
    return cleaned


print("✅ Text cleaning functions defined:")
print("   - clean_text_field(): General text cleaning")
print("   - normalize_persian_text(): Persian-specific normalization")
print("   - normalize_english_text(): English-specific normalization")

✅ Text cleaning functions defined:
   - clean_text_field(): General text cleaning
   - normalize_persian_text(): Persian-specific normalization
   - normalize_english_text(): English-specific normalization


In [19]:
print("=" * 80)
print("CLEANING BASE_PRODUCTS TABLE")
print("=" * 80)

# Create a copy to avoid modifying original
base_products_clean = base_products.copy()

print(f"\n📊 Original shape: {base_products_clean.shape}")

# Step 1: Clean text fields
print("\n1️⃣ Cleaning text fields...")

# Get placeholder tokens from config (with fallback)
placeholder_tokens = config.get('cleaning', {}).get('placeholder_tokens', 
                                                     ['none', 'null', 'nan', 'n/a', 'na', '-', '--', 'unknown', '?'])

# Persian name (critical field - must not be empty)
base_products_clean['persian_name'] = clean_text_field(
    base_products_clean['persian_name'], 
    placeholder_tokens=placeholder_tokens,
    convert_to_none=True
)
base_products_clean['persian_name'] = normalize_persian_text(base_products_clean['persian_name'])

# Image URL
base_products_clean['image_url'] = clean_text_field(
    base_products_clean['image_url'],
    placeholder_tokens=placeholder_tokens,
    convert_to_none=True
)

# Remove english_name column (92% missing - not useful)
if 'english_name' in base_products_clean.columns:
    base_products_clean = base_products_clean.drop(columns=['english_name'])
    print(f"   ✅ Removed english_name column (92% missing)")

# Count missing values
missing_persian = base_products_clean['persian_name'].isna().sum()
missing_image = base_products_clean['image_url'].isna().sum()

print(f"   Persian name missing: {missing_persian:,} ({missing_persian/len(base_products_clean)*100:.3f}%)")
print(f"   Image URL missing: {missing_image:,} ({missing_image/len(base_products_clean)*100:.3f}%)")

# Step 2: Remove products without persian_name (critical field)
print("\n2️⃣ Removing products without persian_name...")
before_removal = len(base_products_clean)
base_products_clean = base_products_clean[base_products_clean['persian_name'].notna()].copy()
removed = before_removal - len(base_products_clean)
print(f"   Removed {removed:,} products without persian_name")

# Step 3: Add flag for products without images
print("\n3️⃣ Adding flag for products without images...")
base_products_clean['has_image'] = base_products_clean['image_url'].notna()
print(f"   Products with images: {base_products_clean['has_image'].sum():,} ({base_products_clean['has_image'].sum()/len(base_products_clean)*100:.2f}%)")
print(f"   Products without images: {(~base_products_clean['has_image']).sum():,} ({(~base_products_clean['has_image']).sum()/len(base_products_clean)*100:.2f}%)")

print(f"\n✅ Base products cleaned: {base_products_clean.shape}")
print(f"   Removed: {original_shapes['base_products'][0] - base_products_clean.shape[0]:,} rows")

CLEANING BASE_PRODUCTS TABLE

📊 Original shape: (1022298, 8)

1️⃣ Cleaning text fields...
   ✅ Removed english_name column (92% missing)
   Persian name missing: 2 (0.000%)
   Image URL missing: 3,341 (0.327%)

2️⃣ Removing products without persian_name...
   Removed 2 products without persian_name

3️⃣ Adding flag for products without images...
   Products with images: 1,018,955 (99.67%)
   Products without images: 3,341 (0.33%)

✅ Base products cleaned: (1022296, 8)
   Removed: 2 rows


In [9]:
def parse_extra_features(value):
    """
    Parse extra_features field into a dictionary.
    Handles various formats: dict, JSON string, empty, etc.
    """
    if pd.isna(value):
        return {}
    
    # Already a dict
    if isinstance(value, dict):
        return value
    
    # Try to parse as JSON string
    if isinstance(value, str):
        value = value.strip()
        if value == '' or value.lower() in ['null', 'none', '{}', '[]']:
            return {}
        try:
            parsed = json.loads(value)
            if isinstance(parsed, dict):
                return parsed
            elif isinstance(parsed, list):
                # Convert list to dict if possible
                return {'items': parsed}
            else:
                return {}
        except (json.JSONDecodeError, ValueError):
            # If JSON parsing fails, return as single key-value
            return {'raw_text': value}
    
    return {}


print("=" * 80)
print("PARSING extra_features FIELD")
print("=" * 80)

# Parse extra_features
print("\n📦 Parsing extra_features JSON...")
base_products_clean['extra_features_parsed'] = base_products_clean['extra_features'].apply(parse_extra_features)

# Count non-empty features
non_empty = base_products_clean['extra_features_parsed'].apply(lambda x: len(x) > 0).sum()
print(f"   Products with features: {non_empty:,} ({non_empty/len(base_products_clean)*100:.2f}%)")
print(f"   Products without features: {len(base_products_clean) - non_empty:,} ({(len(base_products_clean) - non_empty)/len(base_products_clean)*100:.2f}%)")

# Sample parsed features
print("\n📋 Sample parsed features:")
sample_with_features = base_products_clean[base_products_clean['extra_features_parsed'].apply(lambda x: len(x) > 0)].head(3)
for idx, row in sample_with_features.iterrows():
    print(f"\n   Product: {row['persian_name']}")
    print(f"   Features: {row['extra_features_parsed']}")

print("\n✅ extra_features parsed successfully")

PARSING extra_features FIELD

📦 Parsing extra_features JSON...
   Products with features: 1,022,296 (100.00%)
   Products without features: 0 (0.00%)

📋 Sample parsed features:

   Product: فرش 700شانه اُپال فیروزه ای
   Features: {'meterage': ['12 m', '9 m', '6 m'], 'originality': '', 'number_combs': '700 شانه', 'stock_status': 'new', 'carpet_density': '2550', 'background_color': ['فیروزه ای']}

   Product: صندلی تاب مدل chioco
   Features: {'originality': '', 'stock_status': 'new'}

   Product: بشقاب مجموعه کیک یکپارچهسازی با سیستمعامل 13 قطعه 6 رنگ 29.03.2025 English Home
   Features: {'size2': '', 'originality': '', 'piece_count': '13 pieces', 'stock_status': 'new'}

✅ extra_features parsed successfully


In [25]:
print("=" * 80)
print("CLEANING MEMBERS TABLE")
print("=" * 80)

# Create a copy
members_clean = members.copy()

print(f"\n📊 Original shape: {members_clean.shape}")

# Step 1: Remove members with invalid base_random_key (orphaned products)
print("\n1️⃣ Removing orphaned products (invalid base_random_key)...")
valid_base_keys = set(base_products_clean['random_key'])
before_orphan_removal = len(members_clean)
members_clean = members_clean[members_clean['base_random_key'].isin(valid_base_keys)].copy()
orphan_removed = before_orphan_removal - len(members_clean)
print(f"   Removed {orphan_removed:,} orphaned products")

# Step 2: Clean price data
print("\n2️⃣ Cleaning price data...")

# Get price thresholds from config (with fallback)
price_min = config.get('cleaning', {}).get('price_min', 1000)
price_max = config.get('cleaning', {}).get('price_max', 10_000_000_000)

# Statistics before cleaning
print(f"\n   Before cleaning:")
print(f"   - Total products: {len(members_clean):,}")
print(f"   - Mean price: {members_clean['price'].mean():,.0f} Toman")
print(f"   - Median price: {members_clean['price'].median():,.0f} Toman")
print(f"   - Min price: {members_clean['price'].min():,.0f} Toman")
print(f"   - Max price: {members_clean['price'].max():,.0f} Toman")

# Count problematic prices
zero_prices = (members_clean['price'] == 0).sum()
negative_prices = (members_clean['price'] < 0).sum()
null_prices = members_clean['price'].isna().sum()
very_low = (members_clean['price'] < price_min).sum()
very_high = (members_clean['price'] > price_max).sum()

print(f"\n   Problematic prices:")
print(f"   - Zero prices: {zero_prices:,} ({zero_prices/len(members_clean)*100:.2f}%)")
print(f"   - Negative prices: {negative_prices:,} ({negative_prices/len(members_clean)*100:.2f}%)")
print(f"   - Null prices: {null_prices:,} ({null_prices/len(members_clean)*100:.2f}%)")
print(f"   - Very low (<{price_min:,}): {very_low:,} ({very_low/len(members_clean)*100:.2f}%)")
print(f"   - Very high (>{price_max:,}): {very_high:,} ({very_high/len(members_clean)*100:.2f}%)")

# Remove invalid prices
before_price_cleaning = len(members_clean)
members_clean = members_clean[
    (members_clean['price'].notna()) &
    (members_clean['price'] >= price_min) &
    (members_clean['price'] <= price_max)
].copy()
price_removed = before_price_cleaning - len(members_clean)
print(f"\n   Removed {price_removed:,} products with invalid prices")

# Statistics after cleaning
print(f"\n   After cleaning:")
print(f"   - Total products: {len(members_clean):,}")
print(f"   - Mean price: {members_clean['price'].mean():,.0f} Toman")
print(f"   - Median price: {members_clean['price'].median():,.0f} Toman")
print(f"   - Min price: {members_clean['price'].min():,.0f} Toman")
print(f"   - Max price: {members_clean['price'].max():,.0f} Toman")

# Step 3: Remove duplicates (same random_key should be unique)
print("\n3️⃣ Removing duplicate member products...")
before_dedup = len(members_clean)
members_clean = members_clean.drop_duplicates(subset=['random_key']).copy()
dedup_removed = before_dedup - len(members_clean)
print(f"   Removed {dedup_removed:,} duplicate products")

print(f"\n✅ Members table cleaned: {members_clean.shape}")
print(f"   Total removed: {original_shapes['members'][0] - members_clean.shape[0]:,} rows")

CLEANING MEMBERS TABLE

📊 Original shape: (1948665, 4)

1️⃣ Removing orphaned products (invalid base_random_key)...
   Removed 145 orphaned products

2️⃣ Cleaning price data...

   Before cleaning:
   - Total products: 1,948,520
   - Mean price: 154,379,541,844 Toman
   - Median price: 1,700,000 Toman
   - Min price: 0 Toman
   - Max price: 199,000,000,199,000,000 Toman

   Problematic prices:
   - Zero prices: 2 (0.00%)
   - Negative prices: 0 (0.00%)
   - Null prices: 0 (0.00%)
   - Very low (<1,000): 691 (0.04%)
   - Very high (>10,000,000,000): 86 (0.00%)

   Removed 777 products with invalid prices

   After cleaning:
   - Total products: 1,947,743
   - Mean price: 8,274,553 Toman
   - Median price: 1,700,000 Toman
   - Min price: 1,000 Toman
   - Max price: 9,857,600,000 Toman

3️⃣ Removing duplicate member products...
   Removed 0 duplicate products

✅ Members table cleaned: (1947743, 4)
   Total removed: 922 rows


In [15]:
print("=" * 80)
print("CLEANING REFERENCE TABLES")
print("=" * 80)

# Get placeholder tokens from config (with fallback)
placeholder_tokens = config.get('cleaning', {}).get('placeholder_tokens', 
                                                     ['none', 'null', 'nan', 'n/a', 'na', '-', '--', 'unknown', '?'])

# Categories
print("\n📂 Cleaning categories...")
categories_clean = categories.copy()
categories_clean['title'] = clean_text_field(categories_clean['title'], 
                                             placeholder_tokens=placeholder_tokens,
                                             convert_to_none=True)
categories_clean['title'] = normalize_persian_text(categories_clean['title'])
categories_clean = categories_clean.drop_duplicates(subset=['id']).copy()
print(f"   ✅ Categories: {categories_clean.shape}")

# Brands
print("\n🏷️  Cleaning brands...")
brands_clean = brands.copy()
brands_clean['title'] = clean_text_field(brands_clean['title'],
                                         placeholder_tokens=placeholder_tokens,
                                         convert_to_none=True)
# Brands can be in Persian or English, apply both normalizations
brands_clean['title'] = normalize_persian_text(brands_clean['title'])
brands_clean['title'] = normalize_english_text(brands_clean['title'])
brands_clean = brands_clean.drop_duplicates(subset=['id']).copy()
print(f"   ✅ Brands: {brands_clean.shape}")

# Cities
print("\n🏙️  Cleaning cities...")
cities_clean = cities.copy()
cities_clean['name'] = clean_text_field(cities_clean['name'],
                                        placeholder_tokens=placeholder_tokens,
                                        convert_to_none=True)
cities_clean['name'] = normalize_persian_text(cities_clean['name'])
cities_clean = cities_clean.drop_duplicates(subset=['id']).copy()
print(f"   ✅ Cities: {cities_clean.shape}")

# Shops
print("\n🏪 Cleaning shops...")
shops_clean = shops.copy()
# Remove shops that reference non-existent cities
valid_city_ids = set(cities_clean['id'])
before_shop_cleaning = len(shops_clean)
shops_clean = shops_clean[shops_clean['city_id'].isin(valid_city_ids)].copy()
shops_clean = shops_clean.drop_duplicates(subset=['id']).copy()
shop_removed = before_shop_cleaning - len(shops_clean)
print(f"   Removed {shop_removed:,} shops with invalid city_id")
print(f"   ✅ Shops: {shops_clean.shape}")

print("\n✅ All reference tables cleaned")

CLEANING REFERENCE TABLES

📂 Cleaning categories...
   ✅ Categories: (746, 3)

🏷️  Cleaning brands...
   ✅ Brands: (2025, 2)

🏙️  Cleaning cities...
   ✅ Cities: (651, 2)

🏪 Cleaning shops...
   Removed 0 shops with invalid city_id
   ✅ Shops: (23342, 4)

✅ All reference tables cleaned


In [16]:
print("=" * 80)
print("CLEANING USER INTERACTION TABLES")
print("=" * 80)

# Get placeholder tokens from config (with fallback)
placeholder_tokens = config.get('cleaning', {}).get('placeholder_tokens', 
                                                     ['none', 'null', 'nan', 'n/a', 'na', '-', '--', 'unknown', '?'])

# Searches
print("\n🔍 Cleaning searches...")
searches_clean = searches.copy()

# Clean query text
searches_clean['query'] = clean_text_field(searches_clean['query'],
                                           placeholder_tokens=placeholder_tokens,
                                           convert_to_none=True)
searches_clean['query'] = normalize_persian_text(searches_clean['query'])

# Remove searches with empty queries
before_query_removal = len(searches_clean)
searches_clean = searches_clean[searches_clean['query'].notna()].copy()
query_removed = before_query_removal - len(searches_clean)
print(f"   Removed {query_removed:,} searches with empty queries")

# Remove duplicates
before_dedup = len(searches_clean)
searches_clean = searches_clean.drop_duplicates(subset=['id']).copy()
dedup_removed = before_dedup - len(searches_clean)
print(f"   Removed {dedup_removed:,} duplicate searches")

print(f"   ✅ Searches: {searches_clean.shape}")

# Base Views
print("\n👁️  Cleaning base_views...")
base_views_clean = base_views.copy()

# Remove views referencing non-existent searches
valid_search_ids = set(searches_clean['id'])
before_view_cleaning = len(base_views_clean)
base_views_clean = base_views_clean[base_views_clean['search_id'].isin(valid_search_ids)].copy()
view_removed = before_view_cleaning - len(base_views_clean)
print(f"   Removed {view_removed:,} views with invalid search_id")

# Remove views referencing non-existent base products
valid_base_keys = set(base_products_clean['random_key'])
before_bp_removal = len(base_views_clean)
base_views_clean = base_views_clean[base_views_clean['base_product_rk'].isin(valid_base_keys)].copy()
bp_removed = before_bp_removal - len(base_views_clean)
print(f"   Removed {bp_removed:,} views with invalid base_product_rk")

# Remove duplicates
before_dedup = len(base_views_clean)
base_views_clean = base_views_clean.drop_duplicates(subset=['id']).copy()
dedup_removed = before_dedup - len(base_views_clean)
print(f"   Removed {dedup_removed:,} duplicate views")

print(f"   ✅ Base views: {base_views_clean.shape}")

# Final Clicks
print("\n🖱️  Cleaning final_clicks...")
final_clicks_clean = final_clicks.copy()

# Remove clicks referencing non-existent base views
valid_view_ids = set(base_views_clean['id'])
before_click_cleaning = len(final_clicks_clean)
final_clicks_clean = final_clicks_clean[final_clicks_clean['base_view_id'].isin(valid_view_ids)].copy()
click_removed = before_click_cleaning - len(final_clicks_clean)
print(f"   Removed {click_removed:,} clicks with invalid base_view_id")

# Remove clicks referencing non-existent shops
valid_shop_ids = set(shops_clean['id'])
before_shop_removal = len(final_clicks_clean)
final_clicks_clean = final_clicks_clean[final_clicks_clean['shop_id'].isin(valid_shop_ids)].copy()
shop_click_removed = before_shop_removal - len(final_clicks_clean)
print(f"   Removed {shop_click_removed:,} clicks with invalid shop_id")

# Remove duplicates
before_dedup = len(final_clicks_clean)
final_clicks_clean = final_clicks_clean.drop_duplicates(subset=['id']).copy()
dedup_removed = before_dedup - len(final_clicks_clean)
print(f"   Removed {dedup_removed:,} duplicate clicks")

print(f"   ✅ Final clicks: {final_clicks_clean.shape}")

print("\n✅ All user interaction tables cleaned")

CLEANING USER INTERACTION TABLES

🔍 Cleaning searches...
   Removed 0 searches with empty queries
   Removed 0 duplicate searches
   ✅ Searches: (588347, 9)

👁️  Cleaning base_views...
   Removed 0 views with invalid search_id
   Removed 12 views with invalid base_product_rk
   Removed 0 duplicate views
   ✅ Base views: (199904, 4)

🖱️  Cleaning final_clicks...
   Removed 3 clicks with invalid base_view_id
   Removed 0 clicks with invalid shop_id
   Removed 0 duplicate clicks
   ✅ Final clicks: (17368, 4)

✅ All user interaction tables cleaned


In [17]:
print("=" * 80)
print("DATA CLEANING SUMMARY")
print("=" * 80)

# Create summary dataframe
summary_data = []

datasets_comparison = [
    ('base_products', original_shapes['base_products'], base_products_clean.shape),
    ('members', original_shapes['members'], members_clean.shape),
    ('searches', original_shapes['searches'], searches_clean.shape),
    ('base_views', original_shapes['base_views'], base_views_clean.shape),
    ('final_clicks', original_shapes['final_clicks'], final_clicks_clean.shape),
    ('shops', original_shapes['shops'], shops_clean.shape),
    ('categories', original_shapes['categories'], categories_clean.shape),
    ('brands', original_shapes['brands'], brands_clean.shape),
    ('cities', original_shapes['cities'], cities_clean.shape),
]

for name, orig_shape, clean_shape in datasets_comparison:
    rows_removed = orig_shape[0] - clean_shape[0]
    pct_removed = (rows_removed / orig_shape[0] * 100) if orig_shape[0] > 0 else 0
    summary_data.append({
        'Dataset': name,
        'Original Rows': f"{orig_shape[0]:,}",
        'Cleaned Rows': f"{clean_shape[0]:,}",
        'Rows Removed': f"{rows_removed:,}",
        'Removed %': f"{pct_removed:.2f}%",
        'Original Cols': orig_shape[1],
        'Cleaned Cols': clean_shape[1],
    })

summary_df = pd.DataFrame(summary_data)

print("\n📊 CLEANING STATISTICS:")
print(summary_df.to_string(index=False))

# Key data quality metrics
print("\n\n📈 KEY DATA QUALITY METRICS:")
print("-" * 80)
print(f"Base Products:")
print(f"  - Products with persian_name: {base_products_clean['persian_name'].notna().sum():,} (100.00%)")
print(f"  - Products with images: {base_products_clean['has_image'].sum():,} ({base_products_clean['has_image'].sum()/len(base_products_clean)*100:.2f}%)")
print(f"  - Products with extra_features: {base_products_clean['extra_features_parsed'].apply(lambda x: len(x) > 0).sum():,} ({base_products_clean['extra_features_parsed'].apply(lambda x: len(x) > 0).sum()/len(base_products_clean)*100:.2f}%)")

print(f"\nMembers (Shop Products):")
print(f"  - Valid prices: {len(members_clean):,} (100.00%)")
print(f"  - Price range: {members_clean['price'].min():,.0f} - {members_clean['price'].max():,.0f} Toman")
print(f"  - Mean price: {members_clean['price'].mean():,.0f} Toman")
print(f"  - Median price: {members_clean['price'].median():,.0f} Toman")

print(f"\nSearches:")
print(f"  - Searches with queries: {searches_clean['query'].notna().sum():,} (100.00%)")
print(f"  - Unique queries: {searches_clean['query'].nunique():,}")

print(f"\nReferential Integrity:")
print(f"  - All members link to valid base products: ✅")
print(f"  - All base_views link to valid searches: ✅")
print(f"  - All base_views link to valid base products: ✅")
print(f"  - All final_clicks link to valid base_views: ✅")
print(f"  - All final_clicks link to valid shops: ✅")
print(f"  - All shops link to valid cities: ✅")

print("\n✅ Data cleaning completed successfully!")

DATA CLEANING SUMMARY

📊 CLEANING STATISTICS:
      Dataset Original Rows Cleaned Rows Rows Removed Removed %  Original Cols  Cleaned Cols
base_products     1,022,298    1,022,296            2     0.00%              8             9
      members     1,948,665    1,948,015          650     0.03%              4             4
     searches       588,347      588,347            0     0.00%              9             9
   base_views       199,916      199,904           12     0.01%              4             4
 final_clicks        17,371       17,368            3     0.02%              4             4
        shops        23,342       23,342            0     0.00%              4             4
   categories           746          746            0     0.00%              3             3
       brands         2,025        2,025            0     0.00%              2             2
       cities           651          651            0     0.00%              2             2


📈 KEY DATA QUALITY MET

In [28]:
print("=" * 80)
print("SAVING CLEANED DATA")
print("=" * 80)

# Save cleaned datasets
output_path = '../Data/processed'

print(f"\n💾 Saving to: {output_path}")
print("-" * 80)

# Save each table
base_products_clean.to_parquet('../Data/processed/base_products_clean.parquet', index=False)
print(f"✅ Saved: base_products_clean.parquet ({base_products_clean.shape[0]:,} rows)")

members_clean.to_parquet('../Data/processed/members_clean.parquet', index=False)
print(f"✅ Saved: members_clean.parquet ({members_clean.shape[0]:,} rows)")

searches_clean.to_parquet('../Data/processed/searches_clean.parquet', index=False)
print(f"✅ Saved: searches_clean.parquet ({searches_clean.shape[0]:,} rows)")

base_views_clean.to_parquet('../Data/processed/base_views_clean.parquet', index=False)
print(f"✅ Saved: base_views_clean.parquet ({base_views_clean.shape[0]:,} rows)")

final_clicks_clean.to_parquet('../Data/processed/final_clicks_clean.parquet', index=False)
print(f"✅ Saved: final_clicks_clean.parquet ({final_clicks_clean.shape[0]:,} rows)")

shops_clean.to_parquet('../Data/processed/shops_clean.parquet', index=False)
print(f"✅ Saved: shops_clean.parquet ({shops_clean.shape[0]:,} rows)")

categories_clean.to_parquet('../Data/processed/categories_clean.parquet', index=False)
print(f"✅ Saved: categories_clean.parquet ({categories_clean.shape[0]:,} rows)")

brands_clean.to_parquet('../Data/processed/brands_clean.parquet', index=False)
print(f"✅ Saved: brands_clean.parquet ({brands_clean.shape[0]:,} rows)")

cities_clean.to_parquet('../Data/processed/cities_clean.parquet', index=False)
print(f"✅ Saved: cities_clean.parquet ({cities_clean.shape[0]:,} rows)")

# Save cleaning metadata
metadata = {
    'cleaning_date': datetime.now().isoformat(),
    'random_seed': RANDOM_SEED,
    'config': config,
    'original_shapes': {k: list(v) for k, v in original_shapes.items()},
    'cleaned_shapes': {
        'base_products': list(base_products_clean.shape),
        'members': list(members_clean.shape),
        'searches': list(searches_clean.shape),
        'base_views': list(base_views_clean.shape),
        'final_clicks': list(final_clicks_clean.shape),
        'shops': list(shops_clean.shape),
        'categories': list(categories_clean.shape),
        'brands': list(brands_clean.shape),
        'cities': list(cities_clean.shape),
    },
    'summary': {
        'total_rows_removed': sum([
            original_shapes['base_products'][0] - base_products_clean.shape[0],
            original_shapes['members'][0] - members_clean.shape[0],
            original_shapes['searches'][0] - searches_clean.shape[0],
            original_shapes['base_views'][0] - base_views_clean.shape[0],
            original_shapes['final_clicks'][0] - final_clicks_clean.shape[0],
            original_shapes['shops'][0] - shops_clean.shape[0],
            original_shapes['categories'][0] - categories_clean.shape[0],
            original_shapes['brands'][0] - brands_clean.shape[0],
            original_shapes['cities'][0] - cities_clean.shape[0],
        ])
    }
}

with open('../Data/processed/cleaning_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)
print(f"\n✅ Saved: cleaning_metadata.json")

print("\n" + "=" * 80)
print("✅ ALL DATA SAVED SUCCESSFULLY!")
print("=" * 80)
print(f"\nNext steps:")
print(f"  1. Use cleaned data for Feature Engineering")
print(f"  2. Extract embeddings from text and images")
print(f"  3. Build RAG retrieval system")
print(f"\nCleaned data location: {output_path}")

SAVING CLEANED DATA

💾 Saving to: ../Data/processed
--------------------------------------------------------------------------------
✅ Saved: base_products_clean.parquet (1,022,296 rows)
✅ Saved: members_clean.parquet (1,947,743 rows)
✅ Saved: searches_clean.parquet (588,347 rows)
✅ Saved: base_views_clean.parquet (199,904 rows)
✅ Saved: final_clicks_clean.parquet (17,368 rows)
✅ Saved: shops_clean.parquet (23,342 rows)
✅ Saved: categories_clean.parquet (746 rows)
✅ Saved: brands_clean.parquet (2,025 rows)
✅ Saved: cities_clean.parquet (651 rows)

✅ Saved: cleaning_metadata.json

✅ ALL DATA SAVED SUCCESSFULLY!

Next steps:
  1. Use cleaned data for Feature Engineering
  2. Extract embeddings from text and images
  3. Build RAG retrieval system

Cleaned data location: ../Data/processed
